In [ ]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier,BaggingClassifier,StackingClassifier,HistGradientBoostingClassifier,GradientBoostingClassifier

from sklearn.ensemble import StackingClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
estimators = [('RF', RandomForestClassifier()), ('LR',LogisticRegression()), ('XGB', XGBClassifier()), ('DT', DecisionTreeClassifier()), ('SVM', SVC()), ('KNN', KNeighborsClassifier()),('BC',BaggingClassifier()),
    ('HistGB', HistGradientBoostingClassifier()),('ETree',ExtraTreesClassifier()),('GB',GradientBoostingClassifier()),]

In [ ]:
DPC_K  = pd.read_csv('/content/drive/MyDrive/thesis document/thesis/AVP_data_1/training/DPC1.csv')
DPC_L  = pd.read_csv('/content/drive/MyDrive/thesis document/thesis/AVP_data_1/validation/EXTRACTION V/DPC.csv')
for column in DPC_K.columns:
  DPC_K[column] = pd.to_numeric(DPC_K[column], errors='coerce')
  if column == 'TARGET':
    DPC_K[column] = DPC_K[column].astype('category')

DPC_K.fillna(0, inplace=True)
DPC_K.dtypes

DPC1       float64
DPC2       float64
DPC3       float64
DPC4       float64
DPC5       float64
            ...   
DPC397     float64
DPC398     float64
DPC399     float64
DPC400     float64
TARGET    category
Length: 401, dtype: object

In [ ]:
X_train = DPC_K.drop(['TARGET'], axis=1)
y_train = DPC_K.TARGET
X_test = DPC_L.drop(['TARGET'], axis=1)
y_test = DPC_L.TARGET
DPC_L_Metrics = []
DPC_L_Metrics = pd.DataFrame(DPC_L_Metrics)
DPC_L_Metrics['Classifier'] = 'Classifier'
DPC_L_Metrics['Accuracy'] = 'Accuracy'
DPC_L_Metrics['mcc'] = 'mcc'
DPC_L_Metrics['auc'] = 'auc'
DPC_L_Metrics['sensitivity'] = 'sensitivity'
DPC_L_Metrics['specificity'] = 'specificity'
cv = KFold(n_splits=10, random_state=1, shuffle=True)
base_model = [('RT', RandomForestClassifier(n_estimators = 450, max_depth = 9)),
             ('GB',GradientBoostingClassifier(max_depth = 9, n_estimators = 450, learning_rate = 0.5)),]

meta_learner = LogisticRegression()
# create model
models = [LogisticRegression(),
          RandomForestClassifier(n_estimators = 450, max_depth = 9),
          SVC(),
          XGBClassifier(n_estimators = 450,max_depth = 9, base_score = 0.7, learning_rate = 0.5),
          DecisionTreeClassifier(max_depth = 20),
          KNeighborsClassifier(n_neighbors=1),
          BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators = 450, max_samples = 0.5, max_features = 0.5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 15),
          HistGradientBoostingClassifier(random_state=42, max_leaf_nodes=4),
          GradientBoostingClassifier(max_depth = 9, n_estimators = 450, learning_rate = 0.5),
          StackingClassifier(estimators= base_model,final_estimator=meta_learner)]


for model in models:
  # evaluate model
  model.fit(X_train, y_train)
  y_pred = model.predict(y_test)
  cm1 = confusion_matrix(y_test, y_pred)
  # report performance
  Accuracy = accuracy_score(y_test, y_pred)
  mcc = matthews_corrcoef(y_test, y_pred)
  auc = roc_auc_score(y_test, y_pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  DPC_L_Metrics.loc[len(DPC_L_Metrics.index)] = [model,Accuracy, mcc,auc, sensitivity, specificity]

print(DPC_L_Metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


ValueError: ignored